<a href="https://colab.research.google.com/github/Sarthak-125/PINN_2/blob/main/PINN_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [4]:
import numpy as np
import tensorflow as  tf
import random

BC u(t,1) = u(t,-1) = 0

Collocation point creation



In [5]:
n_c = 5000  #collocation points

def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.uniform(start, end)) 
  
    return res

x_c = Rand(-1, 1, n_c)
t_c = Rand(0, 1, n_c)
x_c = tf.cast(x_c ,dtype='float64')
t_c = tf.cast(t_c ,dtype='float64')
input_c = tf.stack((x_c, t_c), axis=1)
x_bc_u = np.ones((n_c,), dtype='float64')
x_bc_b = - x_bc_u
x_bc_u = tf.cast(x_bc_u, dtype='float64')
x_bc_b = tf.cast(x_bc_b, dtype='float64')
input_bcu = tf.stack((x_bc_u, t_c), axis = 1)
input_bcb = tf.stack((x_bc_b, t_c), axis = 1)
input_bc = tf.concat((input_bcu, input_bcb), axis=0)
input_f = tf.concat((input_c, input_bc), axis=0)
x_f = input_f[:,0:1]
t_f = input_f[:,1:2]
y_bc = 0
out_bc = (y_bc* np.ones((2*n_c,1), dtype='float64'))
out_bc = tf.cast(out_bc, dtype='float64')

In [10]:
print(input_bc[:,0:1])

tf.Tensor(
[[ 1.]
 [ 1.]
 [ 1.]
 ...
 [-1.]
 [-1.]
 [-1.]], shape=(10000, 1), dtype=float64)


Architecture ANN

In [7]:
x_in = tf.keras.layers.Input(shape=(1,1), name="Position")
t_in = tf.keras.layers.Input(shape=(1,1), name="Time")
IN = tf.keras.layers.Concatenate(axis=-1, name="Input")([x_in, t_in])
dense1=(tf.keras.layers.Dense(50,activation='tanh'))(IN)
dense2=(tf.keras.layers.Dense(50,activation='tanh'))(dense1)
dense3=(tf.keras.layers.Dense(50,activation='tanh'))(dense2)
dense4=(tf.keras.layers.Dense(50,activation='tanh'))(dense3)
dense5=(tf.keras.layers.Dense(50,activation='tanh'))(dense4)
output=(tf.keras.layers.Dense(1,activation='tanh'))(dense5)
pinn1 = tf.keras.Model(inputs=[x_in,t_in], outputs=output, name="pinn1")

Custom Loss function

In [11]:
def custom_loss_1(arch, x, t):
  def loss(u_train, u_pred):
    with tf.GradientTape(persistent=True) as t1:
      t1.watch(x)
      t1.watch(t)
      with tf.GradientTape() as t2:
        t2.watch(x)
        u = arch([x, t])
        u_x = t2.gradient(u, x)
      u_t = t1.gradient(u, t)
      u_xx = t1.gradient(u_x, x)
    u_pred = tf.cast(u_pred, dtype='float64')
    u_train = tf.cast(u_train, dtype='float64')      
    f = u_t + u*u_x - (0.01/np.pi)*u_xx
    loss_res = tf.reduce_mean(tf.square(f))
    loss_2 = tf.reduce_mean(tf.square(u_pred-u_bc))
    return loss_res + loss_2
  return loss


loss_fn = custom_loss_1(pinn1, x_f, t_f)
pinn1.compile(optimizer='adam', loss=loss_fn, metrics=['mse'])
pinn1.fit([input_bc[:,0:1], input_bc[:,1:2]], out_bc, batch_size=32, epochs=1000)

Epoch 1/1000


TypeError: ignored